### Shape Quantification

This Notebook has been used to make the shape quantification using the csv file exported from Fiji's MorhpolibJ plugin

Path to the MorphlibJ csv file

In [ ]:
filename = "T0/T0-sizeOpen-lbl-killBorders-morpho"

Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Read the CSV file

In [ ]:
df = pd.read_csv(filename+".csv",delimiter=",")

### Compute Shape Descriptors (Elongation and Flatness) in 3D

According to "In situ 3D quantification of the evolution of creep cavity size, shape,
and spatial orientation using synchrotron X-ray tomography" Isaac et al - Materials Science and Engineering - 2007

In [ ]:
a = df["Elli.R1"]
b = df["Elli.R2"]
c = df["Elli.R3"]

In [ ]:
elon=[]
flat =[]
for i in range(a.shape[0]):
    L  = np.max([a[i],b[i],c[i]],axis=0)
    T  = np.min([a[i],b[i],c[i]],axis=0)
    B  = np.median([a[i],b[i],c[i]],axis=0)
      
    e = L / (0.5*(B+T))
    f = B/T
    elon.append(e)
    flat.append(f)
    

Thresholds for Spheric, Rod, Elliptic shapes

In [ ]:
spheric_indix = np.where((np.array(elon)<=1.3) & (np.array(flat)<=1.3))[0]
elong_indix   = np.where(np.array(elon)>2.5)[0]
elli_indix = np.array(list(set(list(range(len(elon))))-set(list(spheric_indix))-set(list(elong_indix))))

Scatter with classes

In [ ]:

fig = plt.figure(figsize=(5,5),dpi=300)
plt.scatter(elon,flat,c="darkmagenta",s=10)
plt.scatter(np.array(elon)[spheric_indix],np.array(flat)[spheric_indix],c="#cccccc", edgecolor="black",s=10)
plt.scatter(np.array(elon)[elong_indix],np.array(flat)[elong_indix],c='#FF9933', s=10)

plt.xscale("log", basex=2)
plt.xlim([0.9,20])
plt.ylim([0.8, 5.6])
plt.ylabel('Flatness (B/T)')
plt.xlabel('Elongation (2L/(B+T))')
plt.legend(['Ellipse','Spheric','Rod'])

plt.savefig(filename + '_elonflat.eps', format="eps")
plt.show()


### Shape characterization and volume correlation

Violin plot Volume versus Shape

In [ ]:
def drawViolinPlot(axis, xlabel, xticks, xticklabels, ylabel, bandwidth, title):
    axis.set_xlabel(xlabel);
    axis.set_xticks(xticks);
    axis.set_xticklabels(xticklabels);
    axis.set_ylabel(ylabel);
    axis.violinplot(sequences, showmeans=True, showmedians=True, bw_method=bandwidth);
    axis.set_title(title);
    
sequences = [
     np.log2(df["Volume"][spheric_indix].values),
     np.log2(df["Volume"][elong_indix].values),
     np.log2(df["Volume"][elli_indix].values)

]

In [ ]:
fig = plt.figure(figsize=(6,3),dpi=300)
plt.subplots_adjust(hspace=1);
bandwidth = None;
drawViolinPlot(fig.gca(),
              "",
              np.arange(len(sequences)+1),
              ('', 'Sphere', 'Rod','Ellipse'),
              "log2 Volume $(µ^3)$", bandwidth,
              "Shape vs Volume");
plt.show();

Violin plot Ellipse Azimuth Versus Shape 

In [ ]:
       
sequences = [
    np.abs(df["Elli.Azim"][spheric_indix].values),
    np.abs(df["Elli.Azim"][elong_indix].values),
    np.abs(df["Elli.Azim"][elli_indix].values)
]

fig = plt.figure(figsize=(6,3),dpi=300)
plt.subplots_adjust(hspace=1);
bandwidth = None;
drawViolinPlot(fig.gca(),
              "",
              np.arange(len(sequences)+1),
              ('', 'Sphere', 'Rod','Ellipse'),
              "Ellipse Azimuth (degrees)", bandwidth,
              "Shape vs Ellipse Azimuth");
plt.show();

Histograms

In [ ]:
plt.figure(figsize=[10,8],dpi=300)
bins = np.linspace(0, 90, 15)

x1 = np.abs(df["Elli.Azim"][spheric_indix].values)
x2 = np.abs(df["Elli.Azim"][elong_indix].values)
x3 = np.abs(df["Elli.Azim"][elli_indix].values)

weights=[np.zeros_like(x2)+1./x2.shape[0], np.zeros_like(x3)+1./x3.shape[0], np.zeros_like(x1)+1./x1.shape[0]]
colors = ['#FF9933', 'darkmagenta', '#cccccc']
labels=['rod','ellipse','sphere']
plt.hist([x2, x3, x1], bins, color=colors, edgecolor='black', label=labels, weights=weights)

plt.xlabel('azimut angle (degrees)',fontsize=15)
plt.ylabel('mitochondria distribution',fontsize=15)
plt.ylim([0,0.72])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('flight muscle',fontsize=15)
plt.legend(loc='upper right')

plt.savefig(filename+'_volazim_hist.eps', dpi=300, format="eps")
plt.show()


In [ ]:
#display as log2 scale
x1 = np.log2(df["Volume"][spheric_indix])
x2 = np.log2(df["Volume"][elong_indix])
x3 = np.log2(df["Volume"][elli_indix])


In [ ]:
plt.figure(figsize=[15,8], dpi=300)
bins = np.linspace(-3, 6, 20)

plt.hist([x3, x1, x2], bins, edgecolor='black')
#plt.grid(axis='y', alpha=0.75)
plt.xlabel('Volume log2 $(µm^3)$',fontsize=15)
plt.ylabel('Mitochondria number',fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('Leg Muscle',fontsize=15)
#plt.legend(loc='upper left')

plt.show()

In [ ]:
plt.figure(figsize=[10,8],dpi=300)
bins = np.linspace(-3, 6, 15)

colors=['darkmagenta','#cccccc','#FF9933']
labels=['ellipse','sphere','rod']
plt.hist([x3, x1, x2], bins, color=colors, edgecolor='black', label=labels)

plt.xlabel('Volume log2 $(µm^3)$',fontsize=15)
plt.ylabel('Mitochondria number',fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('flight muscle',fontsize=15)
plt.legend(loc='upper left')

plt.savefig(filename+"_volume_ifm_histogram.eps",format="eps")
plt.show()

In [ ]:
plt.figure(figsize=[10,8])
bins = np.linspace(-10, 10, 50)

plt.hist(x3, bins, alpha = 0.3, color='darkmagenta', edgecolor='black', label='ellipse',weights=np.zeros_like(x3) + 1. / x3.shape[0] ) #alpha defines transparency
plt.hist(x2, bins, alpha = 0.3, color='#FF9933', edgecolor='black', label='rod',weights=np.zeros_like(x2) + 1. / x2.shape[0])
plt.hist(x1, bins, alpha = 0.3, color='white', edgecolor='black', label='spheric',weights=np.zeros_like(x1) + 1. / x1.shape[0])
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Volume log2 $(µm^3)$',fontsize=15)
plt.ylabel('Mitochondria distribution',fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('Leg Muscle',fontsize=15)
plt.legend(loc='upper left')

plt.show()

In [ ]:
def hist(x, ax=None, title="", nbins=30):
    cm = plt.cm.get_cmap("plasma")
    ax = ax or plt.gca()
    n, bins, patches = ax.hist(x,color="r",bins=range(0, 50 + nbins))
    n = np.cumsum(n)

    bin_centers = 0.5*(bins[:-1]+bins[1:])

    norm = plt.Normalize(0,50)

    for c, p in zip(bin_centers, patches):
        plt.setp(p, "facecolor", cm(norm(c)))
    plt.title(title)
    #plt.yscale("log")
    plt.xlabel("Volume $(µ^3)$")
    plt.ylabel("#Mitochondria")
    plt.xlim([0,50])



In [ ]:
plt.figure(figsize=(5,2),dpi=150)
x1 = df["Volume"][spheric_indix]
hist(x1,ax=plt.gca(),title="Sphere")
plt.savefig(filename+"_volume_sphere_histogram.tif",format="png")
plt.show()

plt.figure(figsize=(5,2),dpi=150)
x2 = df["Volume"][elong_indix]
hist(x2,ax=plt.gca(),title="Rod")
plt.savefig(filename+"_volume_rod_histogram.tif",format="png")
plt.show()


plt.figure(figsize=(5,2),dpi=150)
x3 = df["Volume"][elli_indix]
hist(x3,ax=plt.gca(),title="Ellipse")
plt.savefig(filename+"_volume_ellipse_histogram.tif",format="png")
plt.show()




Export CSV (contains shape classes)

In [ ]:
shape = [""]*df.shape[0]

for i in spheric_indix:
    shape[i]="spheric"
for i in elli_indix:
    shape[i]="ellipse"
for i in elong_indix:
    shape[i]="rod"
df["shape"]=shape

In [ ]:
df.to_csv('T0/SHAPE-T0-sizeOpen-lbl-killBorders-morpho.csv',sep=',')